In [1]:
import ollama
from pathlib import Path
from typing import List, Dict
import re
from collections import defaultdict
import yaml
from datetime import datetime

# Helper Functions
def sanitize_path_element(element: str) -> str:
    """Sanitize directory and filenames for all OSes"""
    return re.sub(r'[^\w\-_\.]', '', element.replace(' ', '-')).strip('-')

def is_horizontal_rule(line: str) -> bool:
    """Check if a line is a Markdown horizontal rule"""
    return bool(re.match(r'^[-*_]{3,}$', line.strip()))

# Core Processing Functions
def generate_metadata(content: str, model: str) -> Dict:
    """Generate metadata using local LLM"""
    try:
        if not content.strip():
            return {}
            
        prompt = f"""
        Analyze this text and extract key concepts following these rules:
        1. Identify primary concept (PascalCase)
        2. List 2-5 related concepts (PascalCase)
        3. Generate 1-3 tags (lowercase-with-dashes)
        4. Create a 1-sentence summary
        
        Text: {content[:2000]}
        """
        
        response = ollama.generate(
            model=model,
            prompt=prompt,
            format="json",
            options={"temperature": 0.2}
        )
        return response.json().get("response", {})
    except:
        return {}

def generate_ai_content(title: str, concepts: List[str], folder_hierarchy: List[str], model: str) -> str:
    """Generate content using AI with folder context"""
    try:
        context_path = " > ".join(folder_hierarchy)
        prompt = f"""
        Generate comprehensive content for: {title}
        Context Hierarchy: {context_path}
        Include:
        - Core definitions
        - Practical applications
        - Relationships to parent concepts
        - Simple examples
        Use academic tone with Markdown sections
        """
        
        response = ollama.generate(
            model=model,
            prompt=prompt,
            options={"temperature": 0.5}
        )
        return f"> **AI Generated Content**\n{response['response']}"
    except:
        return "> **AI Generation Failed** - Content placeholder"

def merge_moc_content(existing_content: str, new_moc: str) -> str:
    """Merge existing note content with MOC sections"""
    if existing_content.startswith('---'):
        frontmatter_end = existing_content.find('---', 3)
        if frontmatter_end != -1:
            frontmatter = existing_content[:frontmatter_end+3]
            existing_content = existing_content[frontmatter_end+3:].lstrip()
            existing_content = re.sub(r'##+ (Subtopics|Notes)[\s\S]*?(?=##|$)', '', existing_content).strip()
            return f"{frontmatter}\n{existing_content}\n\n{new_moc}"
    return f"{existing_content}\n\n{new_moc}".strip()

# Main Processing Pipeline
def process_note_with_metadata(note_path: Path, output_dir: Path, model: str, notes_root: Path):
    """Process notes with full error handling and MOC merging"""
    try:
        rel_path = note_path.relative_to(notes_root)
        print(f"\n📁 Processing: {rel_path}")
        
        with open(note_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        sections = re.split(r'\n##+ ', content)
        print(f"  🔍 Found {len(sections)} sections")
        
        sanitized_hierarchy = [sanitize_path_element(p) for p in rel_path.parent.parts]
        note_stem = sanitize_path_element(note_path.stem)
        
        output_folder = output_dir.joinpath(*sanitized_hierarchy, note_stem)
        
        # Handle file/folder conflicts
        if output_folder.exists() and output_folder.is_file():
            new_name = f"{note_stem}-{datetime.now().strftime('%Y%m%d%H%M%S')}"
            output_folder = output_dir.joinpath(*sanitized_hierarchy, new_name)
            print(f"  ⚠️  Renamed conflicting file to: {new_name}")
        
        output_folder.mkdir(parents=True, exist_ok=True)
        
        moc_entries = []
        for i, section in enumerate(sections, 1):
            if not section.strip():
                continue
                
            lines = section.split('\n')
            original_title = lines[0].strip('#').strip()
            
            if is_horizontal_rule(original_title) or not original_title:
                print(f"  ⚠️  Skipping invalid section: {original_title}")
                continue
                
            body = '\n'.join(lines[1:])
            print(f"  📝 Section {i}: {original_title}")
            
            try:
                # Metadata generation
                metadata = generate_metadata(f"{original_title}\n\n{body}", model)
                concepts = metadata.get('concepts', [])
                ai_generated = False
                
                # Content generation
                if not body.strip() or not re.search(r'^#+ ', body, flags=re.MULTILINE):
                    print("    🤖 Generating AI content...")
                    body = generate_ai_content(original_title, concepts, sanitized_hierarchy, model)
                    ai_generated = True
                
                # File handling
                sanitized_name = sanitize_path_element(original_title)
                output_path = output_folder / f"{sanitized_name}.md"
                counter = 1
                while output_path.exists():
                    output_path = output_folder / f"{sanitized_name}-{counter}.md"
                    counter += 1
                
                # Frontmatter creation
                frontmatter = {
                    'created': datetime.now().isoformat(),
                    'modified': datetime.now().isoformat(),
                    'source': f"[[{note_stem}]]",
                    'hierarchy': sanitized_hierarchy,
                    'tags': metadata.get('tags', []),
                    'summary': metadata.get('summary', ''),
                    'concepts': concepts,
                    'ai_generated': ai_generated
                }
                yaml_front = yaml.safe_dump(frontmatter, sort_keys=False, allow_unicode=True)
                
                # Note assembly
                note_content = f"---\n{yaml_front}\n---\n\n"
                note_content += f"# {original_title}\n\n"
                note_content += f"## Context Path\n{' > '.join(sanitized_hierarchy)}\n\n" if sanitized_hierarchy else ""
                note_content += "## Content\n" + body + "\n\n"
                note_content += "## Related Concepts\n" + '\n'.join(f"[[{c}]]" for c in concepts[1:])
                
                output_path.write_text(note_content, encoding='utf-8')
                moc_entries.append(output_path)
                print(f"    💾 Saved to: {output_path.relative_to(output_dir)}")
                
            except Exception as e:
                print(f"    ❌ Section processing failed: {str(e)}")
                continue
        
        # MOC creation
        create_folder_moc(output_folder, moc_entries)
    
    except Exception as e:
        print(f"❌ Failed to process {note_path.name}: {str(e)}")

def create_folder_moc(folder_path: Path, entries: List[Path]):
    """Create/update MOC for each folder with merging"""
    moc_name = f"{folder_path.name} MOC.md"
    moc_path = folder_path / moc_name
    
    # Existing content handling
    existing_content = ""
    if moc_path.exists():
        try:
            existing_content = moc_path.read_text(encoding='utf-8')
            print(f"    🔄 Merging with existing MOC content")
        except Exception as e:
            print(f"    ⚠️  Error reading MOC: {str(e)}")
    
    # New MOC content
    new_moc = []
    subfolders = [f for f in folder_path.iterdir() if f.is_dir()]
    if subfolders:
        new_moc.append("## Subtopics")
        new_moc.extend(f"- [[{f.name}/{f.name} MOC]]" for f in subfolders)
    
    notes = [f for f in folder_path.iterdir() 
             if f.is_file() and f != moc_path and f.suffix == '.md']
    if notes:
        new_moc.append("\n## Notes")
        new_moc.extend(f"- [[{f.stem.replace('-', ' ')}]]" for f in notes)
    
    # Merge contents
    final_content = f"# {folder_path.name} Map of Content\n\n" + '\n'.join(new_moc)
    if existing_content:
        final_content = merge_moc_content(existing_content, '\n'.join(new_moc))
    
    # Add backlink
    if folder_path != folder_path.parent:
        parent_moc = folder_path.parent / f"{folder_path.parent.name} MOC.md"
        final_content += f"\n\n⤴️ Back to [[{parent_moc.stem.replace('-', ' ')}]]"
    
    # Write final MOC
    try:
        moc_path.write_text(final_content, encoding='utf-8')
        print(f"    📑 Updated MOC at: {moc_path.relative_to(folder_path.parent)}")
    except Exception as e:
        print(f"    ❌ MOC creation failed: {str(e)}")

# Global Index and Main Execution
def create_global_indices(output_dir: Path):
    """Create hierarchical indices"""
    print("\n📚 Building global indices...")
    index_content = "# Knowledge Hierarchy Index\n\n"
    
    for path in sorted(output_dir.glob("**/* MOC.md")):
        if path.name.startswith('_'):
            continue
            
        relative_path = path.relative_to(output_dir)
        depth = len(relative_path.parent.parts) - 1
        indent = "  " * depth
        display_name = ' '.join(relative_path.parent.name.split('-'))
        
        index_content += f"{indent}- [[{display_name} MOC]]\n"
    
    index_path = output_dir / "_HIERARCHY.md"
    index_path.write_text(index_content, encoding='utf-8')
    print(f"    📖 Global index created at: {index_path}")

if __name__ == "__main__":
    VAULT_ROOT = Path("/home/vikk/Documents/GitHub/College-Notes")
    NOTES_ROOT = VAULT_ROOT / "Notes"
    OUTPUT_DIR = VAULT_ROOT / "Structured_Notes"
    MODEL = "mistral-small:22b"
    
    print("🚀 Starting note processing pipeline")
    print(f"🔧 Using model: {MODEL}")
    print(f"📂 Input directory: {NOTES_ROOT}")
    print(f"📂 Output directory: {OUTPUT_DIR}")
    
    # Process notes with progress tracking
    total = sum(1 for _ in NOTES_ROOT.glob("**/*.md") if "MOC" not in _.name)
    processed = 0
    
    for note_path in NOTES_ROOT.glob("**/*.md"):
        if "MOC" in note_path.name:
            continue
        
        try:
            process_note_with_metadata(note_path, OUTPUT_DIR, MODEL, NOTES_ROOT)
            processed += 1
            print(f"✅ Progress: {processed}/{total} ({processed/total:.1%})")
        except Exception as e:
            print(f"❌ Critical error processing {note_path}, skipping: {str(e)}")
    
    # Final indices
    create_global_indices(OUTPUT_DIR)
    
    print(f"\n🎉 Processing complete! Success: {processed}/{total} notes")
    print(f"🌐 Index available at: {OUTPUT_DIR}/_HIERARCHY.md")

🚀 Starting note processing pipeline
🔧 Using model: mistral-small:22b
📂 Input directory: /home/vikk/Documents/GitHub/College-Notes/Notes
📂 Output directory: /home/vikk/Documents/GitHub/College-Notes/Structured_Notes

📁 Processing: Temp notes.md
  🔍 Found 1 sections
❌ Failed to process Temp notes.md: name 'sanitize_path_element' is not defined
✅ Progress: 1/153 (0.7%)

📁 Processing: Software and System Design/Unit 2.md
  🔍 Found 9 sections
❌ Failed to process Unit 2.md: name 'sanitize_path_element' is not defined
✅ Progress: 2/153 (1.3%)

📁 Processing: Papers/Quantum Computing Paper.md
  🔍 Found 13 sections
❌ Failed to process Quantum Computing Paper.md: name 'sanitize_path_element' is not defined
✅ Progress: 3/153 (2.0%)

📁 Processing: Object Oriented Programming/Object Oriented Programming.md
  🔍 Found 26 sections
❌ Failed to process Object Oriented Programming.md: name 'sanitize_path_element' is not defined
✅ Progress: 4/153 (2.6%)

📁 Processing: IOT/Electrical Saftey.md
  🔍 Found 8 s

NameError: name 'create_global_indices' is not defined